**Processamento da Refined** - Desafio Parte III
- Processamento do arquivo parquet movies da camada Trusted.

In [ ]:
!pip install boto3
!pip install pyspark

In [2]:
import boto3
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Refined").getOrCreate()

In [3]:
aws_access_key_id = ''
aws_secret_access_key = ''
aws_session_token = ''

client = boto3.client('s3', aws_access_key_id = aws_access_key_id,
                  aws_secret_access_key = aws_secret_access_key,
                  aws_session_token = aws_session_token)

In [ ]:
from io import BytesIO

bucket_name = "data-lake-livia"
input_key = "TRT/Movies/movies.parquet"

s3_object = client.get_object(Bucket=bucket_name, Key=input_key)['Body'].read()
parquet = BytesIO(s3_object)
df_pd = pd.read_parquet(parquet)

print(df_pd.info())

In [ ]:
colunas_selecionadas = ['id', 'tituloPincipal', 'anoLancamento', 'genero', 'notaMedia', 'numeroVotos']
df_selecionado = df_pd[colunas_selecionadas]

df_suspense = df_selecionado[df_selecionado['genero'].apply(lambda x: 'Thriller' in x)]

print(df_suspense)

In [6]:
file_name = "suspense.parquet"

file_movies = f'/content/{file_name}'
df_suspense.to_parquet(file_movies, index=False)

In [ ]:
folder_name = "Suspense"

up_file_movies = f'/content/{file_name}'
aws_file_movies = f"RFD/{folder_name}/{file_name}"

try:
    client.upload_file(up_file_movies, bucket_name, aws_file_movies)
    print(f"Sucesso! Arquivo parquet enviado para o S3.")
except Exception as e:
    print(f"Erro durante o upload: {e}")